# Lab session 6: Association Analysis

## Introduction

The purpose of this lab session is to provide you with an opportunity to gain experience in **association analysis**, covered in week 8, using typical Python libraries.

- This lab is the first part of a **two-week assignment** that covers weeks 8 and 9, which is due on **Friday 3rd December 10am**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet will contribute to 5% of your overall grade; questions in the lab sheet for week 9 will cover for another 5% of your overall grade.
- <font color = 'maroon'>The last section of this notebook includes the questions that are assessed towards your final grade.</font> 

This session starts with a tutorial that uses examples to introduce you to the practical knowledge that you will need for the corresponding assignment. We highly recommend that you read the following tutorials if you need a gentler introduction to the libraries that we use:
- [Mlxtend: Apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
- [Numpy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html)
- [Numpy: basic broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- [Matplotlib](https://matplotlib.org/tutorials/introductory/pyplot.html)
- [Seaborn](https://seaborn.pydata.org/tutorial/relational.html)
- [Scikit-learn](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

## Important notes about the assignment: 

- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- You are asked to submit a **report** that should answer the questions specified in the last section of this notebook. The report should be in a **single PDF document** (so **NOT** *doc, docx, notebook* etc). This single PDF document will include your answers to both the week 8 and week 9 labs.
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment3-StudentName-StudentNumber.pdf
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.

## 1. Frequent itemsets

In order to present functionalities for association analysis in Python, we adapt an example from the ``mlxtend`` documentation.

Consider a dataset composed of five transactions. This dataset is represented by a list of five elements, each of which is a list of items bought during a trip to a supermarket.

In [4]:
import numpy as np
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

The library ``mlxtend`` requires that each transaction is represented by a binary vector where each element indicates the presence or absence of a specific item.

The method ``TransactionEncoder.fit_transform`` can be used to convert the dataset created above into this expected format. This method returns a binary matrix (numpy array) where each transaction corresponds to a row and each column corresponds to an item.

In [5]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
print(te_ary)

[[False False False  True False  True  True  True  True False  True]
 [False False  True  True False  True False  True  True False  True]
 [ True False False  True False  True  True False False False False]
 [False  True False False False  True  True False False  True  True]
 [False  True False  True  True  True False False  True False False]]


The item corresponding to each column is stored by the ``TransactionEncoder`` object in a variable called ``columns_``. This variable can be used to create a ``DataFrame`` that conveniently represents the transaction dataset.

In [6]:
import pandas as pd

df = pd.DataFrame(te_ary, columns=te.columns_)
display(df)

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


The ``mlxtend`` function ``apriori`` receives a ``DataFrame`` that represents a transaction dataset and a parameter that specifies the support threshold. This function returns a ``DataFrame`` that contains one row for each frequent itemset. Each row contains a python ``frozenset`` that represents the itemset (by column indices) and a number that represents the support of this itemset.

In [7]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.6)
display(frequent_itemsets)

itemset = frequent_itemsets.loc[5]
print('Itemset: {0}. Support: {1}.'.format(itemset['itemsets'], itemset['support']))

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


Itemset: frozenset({3, 5}). Support: 0.8.


Conveniently, if the parameter ``use_colnames`` is set to ``True``,  the ``mlxtend`` function ``apriori`` may instead return a ``DataFrame`` that represents itemsets by ``frozensets`` of item names.

In [10]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['count'] = frequent_itemsets.support*5
display(frequent_itemsets)

,support,itemsets,count
0,0.8,(Eggs),4.0
1,1.0,(Kidney Beans),5.0
2,0.6,(Milk),3.0
3,0.6,(Onion),3.0
4,0.6,(Yogurt),3.0
5,0.8,"(Kidney Beans, Eggs)",4.0
6,0.6,"(Onion, Eggs)",3.0
7,0.6,"(Kidney Beans, Milk)",3.0
8,0.6,"(Onion, Kidney Beans)",3.0
9,0.6,"(Kidney Beans, Yogurt)",3.0


In [ ]:
count = 0
for x in frequent_itemsets.itemsets:
    for y in list(x):
        if y == 'Onion':
            count += 1
print(count)

In [ ]:
count = 0
for x in frequent_itemsets.itemsets:
    if list(x) == ['Onion', 'Eggs', 'Kidney Beans']:
        count += 1
print(count)

In [ ]:
count = 0
for x in frequent_itemsets.itemsets:
    if 'Kidney Beans' in list(x) and 'Eggs' in list(x):
        count += 1
print(count)

In [ ]:
def kulz_score(df,first,second):
    count1 = 0
    for x in df:
        for y in list(x):
            if y == first:
                count1 += 1
    count2 = 0
    second.insert(0,first)
    for x in df:
        if list(x) == second:
            count2 += 1
    count3 = 0
    for x in df:
        if second[0] in list(x) and second[1] in list(x):
            count3 += 1
    return ((count2/count1)+(count2/count3))/2
kulz_score(frequent_itemsets.itemsets,'Onion',['Eggs','Kidney Beans'])

In [ ]:
def kulz_score(df,first,second):
    count1 = 0
    for x in df:
        for y in list(x):
            if y == first:
                count1 += 1
    count2 = 0
    second.insert(0,first)
    for x in df:
        if list(x) == second:
            count2 += 1
    count3 = 0
    for x in df:
        if second[0] in list(x) and second[1] in list(x):
            count3 += 1
    return ((count2/count1)+(count2/count3))/2
kulz_score(frequent_itemsets.itemsets,'Eggs',['Onion','Kidney Beans'])

In [ ]:
lists = ['Eggs','Kidney Beans']
lists.insert(0,'Onion')
if ['Onion', 'Eggs', 'Kidney Beans'] == lists:
    print('buuts')


Using typical ``pandas`` functionalities, it is easy to include a column in such a ``DataFrame`` to register the number of items in each frequent itemset, which can be used to filter itemsets by length.

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # length of each frozenset
print('Frequent 3-itemsets:')
display(frequent_itemsets[frequent_itemsets['length'] == 3])

It is also easy to create a ``dict`` that maps any frequent itemset (represented by a ``frozenset``) to its support.

In [ ]:
support = {}
for _, row in frequent_itemsets.iterrows():
    support[row['itemsets']] = row['support']

itemset = frozenset(['Onion', 'Eggs'])
print('Itemset: {0}. Support: {1}.'.format(itemset, support[itemset]))

In [ ]:
def kulz_2(df, first):
    count1 = 0
    for _, row in df.iterrows():
            if 'Onion' in row['itemsets']:
                count1 += 1
    return count1
print(kulz_2(frequent_itemsets,['Onion','Eggs']))

## 2. Association rules

The ``mlxtend`` function ``association_rules`` receives a ``DataFrame`` that represents the set of frequent itemsets and returns a ``DataFrame`` that represents strong association rules for a specified confidence threshold. Each row in the resulting  ``DataFrame`` contains an association rule together with some potentially useful measures (we have not covered lift, leverage, or conviction). 

In [ ]:
from mlxtend.frequent_patterns import association_rules

strong_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
display(strong_rules)

## <font color = 'maroon'>Assignment 3 [Part 1 of 2]</font>

Questions 1-6 are pen-and-paper exercises (brief answers and justifications are expected). Questions 7-8 are coding exercises. In all responses, please show your workings (equations, justifications, or code when applicable).

1. What is the advantage of using the Apriori algorithm in comparison with computing the support of every subset of an itemset in order to find the frequent itemsets in a transaction dataset? [0.5 marks out of 5]
2. Let $\mathcal{L}_1$ denote the set of frequent $1$-itemsets. For $k \geq 2$, why must every frequent $k$-itemset be a superset of an itemset in $\mathcal{L}_1$? [0.5 marks out of 5]
3. Let $\mathcal{L}_2 = \{ \{1,2\}, \{1,4\}, \{2, 3\}, \{2, 4\}, \{3, 5\}\}$. Compute the set of candidates $\mathcal{C}_3$ that is obtained by joining every pair of joinable itemsets from $\mathcal{L}_2$. [0.5 marks out of 5]
4. Let $S_1$ denote the support of the association rule $\{ \text{boarding pass, passport} \} \Rightarrow \{ \text{flight} \}$. Let $S_2$ denote the support of the association rule $\{ \text{boarding pass} \} \Rightarrow \{ \text{flight} \}$. What is the relationship between $S_1$ and $S_2$? [0.5 marks out of 5]
5. What is the support of the rule $\{  \} \Rightarrow \{ \text{Eggs} \}$ in the transaction dataset used in Section 1 of this lab notebook? [0.5 marks out of 5]
6. In the transaction dataset used in the tutorial presented above, what is the maximum length of a frequent itemset for a support threshold of 0.2? [0.5 marks out of 5]
7. Implement a function that computes the Kulczynski measure of two itemsets $\mathcal{A}$ and $\mathcal{B}$. Use your function to compute the Kulczynski measure for itemsets $\mathcal{A} = \{\text{Onion}\}$ and $\mathcal{B} = \{\text{Kidney Beans}, \text{Eggs}\}$ in the transaction dataset used in this lab notebook. [1 mark out of 5]
8. Implement a function that computes the imbalance ratio of two itemsets $\mathcal{A}$ and $\mathcal{B}$. Use your function to compute the imbalance ratio for itemsets $\mathcal{A} = \{\text{Onion}\}$ and $\mathcal{B} = \{\text{Kidney Beans}, \text{Eggs}\}$ in the transaction dataset used in this lab notebook. [1 mark out of 5]
